# Часть Б. Игра с тегами

Раздел состоит из частей 3, 4, 5. В 3 - подключается библиотека для обработки именованных сущностей. В 4 - используется ещё один подход для вытаскивания тегов из текста. В 5 - Все анекдоты процессятся и полученные теги записываются в Базу Данных.

## Часть 3. Именованные сущности.
Я тут перепробовала какие-то разные библиотеки и решила (хехе) остановиться на библиотеке от Стенфорда Stanza (вовсе не потому что я только что смотрела Дудя про Силиконовую долину, а там как раз Стенфордский университет!, а потому что она работает и на русском).

### 3.1. Устанавливаем библиотеку, проверяем, как она работает.

In [1]:
# куча проблем на этом этапе

In [18]:
import stanza
stanza.download('ru')

2022-03-29 13:02:13 INFO: Downloading default packages for language: ru (Russian)...


2022-03-29 13:06:48 INFO: Finished downloading models and saved to C:\Users\kseni\stanza_resources.


In [19]:
anecdot = "Лента.ру: \"... США подготовили санкции против элит России на случай «вторжения» на Украину...\"А именно: ВВС США будут жестко бомбить побережье Флориды и Калифорнии. Также будут нанесены точечные удары по некоторым элитным районам Нью-Йоркa."

In [23]:
def take_ne(text):
  nlp = stanza.Pipeline(lang='ru', processors='tokenize,ner')
  doc = nlp(text)
  list_of_entities = []
  for sent in doc.sentences:
    for ent in sent.ents:
      list_of_entities.append(ent.text)
  return list_of_entities
  #print(*[f'entity: {ent.text}\ttype: {ent.type}' for sent in doc.sentences for ent in sent.ents], sep='\n')


In [24]:
take_ne(anecdot)

2022-03-29 13:09:49 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| ner       | wikiner   |

2022-03-29 13:09:49 INFO: Use device: cpu
2022-03-29 13:09:49 INFO: Loading: tokenize
2022-03-29 13:09:49 INFO: Loading: ner
2022-03-29 13:09:51 INFO: Done loading processors!


['Лента.ру',
 'США',
 'России',
 'Украину',
 'ВВС США',
 'Флориды',
 'Калифорнии',
 'Нью-Йоркa']

Хочу для тегов получать именованный сущности в начальной форме, поэтому напишем функцию обработки

In [25]:
from pymorphy2 import MorphAnalyzer # Сам pymorphy
morph = MorphAnalyzer()

In [26]:
def named_entity_tags(text):
  not_normal_form = stanza_nlp(text)
  normal_form = []
  for i in not_normal_form:
    normal_form.append(morph.parse(i)[0].normal_form)
  return normal_form

### Часть 4. NER не идеально, ещё один подход к тегам.

Важно понимать, что библиотека не всесильна, она не всегда достаёт сами сущности, но, что важнее для решаемой задачи - не вытаскивает (как бы от неё это и не требуют, это же не NE) некоторые очень характерные для анекдотов слова.  
(Например, слова **тёща** и **еврей**)

Их не вытащить так "втупую", а иметь в тегах очень хочется, поэтому - попробуем подойти с другой стороны.

**Идея примерно следующая:**  
* Обкачать список тем с сайта
* приведу их в начальную форму
* достающую их из уже лемматизированного текста анекдотов

In [28]:
import requests
from pprint import pprint
from bs4 import BeautifulSoup
session = requests.session()
from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)
import re

In [29]:
full_url = 'http://www.yaxy.ru/anecdote/allteam.html'
# На этом сайте круче набор тем
req = session.get(full_url, headers={'User-Agent': ua.random})
page = req.text
soup = BeautifulSoup(page, 'html.parser')

b = soup.find_all('ul', {'type': 'square'}) 
b[0].text
# При помощи регулярных выражений достаём
c = re.findall(r'про (.*)', b[0].text)
c

['алкашей ',
 'наркоманов ',
 'армию ',
 'Вовочку ',
 'время года ',
 'блондинок ',
 'брюнеток ',
 'девушек ',
 'женщин ',
 'животных ',
 'котов ',
 'жизнь ',
 'Гену ',
 'Чебурашку ',
 'любовь ',
 'жену ',
 'мужа ',
 'мужчин ',
 'парней ',
 'американцев ',
 'армян ',
 'грузин ',
 'дагестанцев ',
 'кавказцев ',
 'казахов ',
 'китайцев ',
 'немцев ',
 'русских ',
 'украинцев ',
 'чеченцев ',
 'охоту ',
 'рыбалку ',
 'Гарри Поттера ',
 'Чака Норриса ',
 'Чапаева ',
 'выборы ',
 'Ленина ',
 'Лукашенко ',
 'Медведева ',
 'Путина ',
 'Януковича ',
 'Ржевского ',
 '1 апреля ',
 '23 февраля ',
 '8 марта ',
 'Новый год ',
 'врачей ',
 'гаишников ',
 'работу ',
 'учителей ',
 'студентов ',
 'зятя ',
 'тещу ',
 'чукчу ',
 'Шерлока Холмса ',
 'школу ',
 'Штирлица ']

In [30]:
my_tags = []
for i in c:
  i = i.rstrip(" ")
  if i[0].isupper():
    my_tags.append(morph.parse(i)[0].normal_form.title()) # Бережно храню..
  else:
    my_tags.append(morph.parse(i)[0].normal_form)
my_tags

['алкаш',
 'наркоман',
 'армия',
 'Вовочка',
 'время года',
 'блондинка',
 'брюнетка',
 'девушка',
 'женщина',
 'животное',
 'кот',
 'жизнь',
 'Ген',
 'Чебурашка',
 'любовь',
 'жена',
 'муж',
 'мужчина',
 'парень',
 'американец',
 'армянин',
 'грузин',
 'дагестанец',
 'кавказец',
 'казах',
 'китаец',
 'немец',
 'русский',
 'украинец',
 'чеченец',
 'охота',
 'рыбалка',
 'Гарри Поттер',
 'Чака Норрис',
 'Чапаев',
 'выборы',
 'Ленин',
 'Лукашенко',
 'Медведев',
 'Путин',
 'Янукович',
 'Ржевский',
 '1 апрель',
 '23 февраль',
 '8 март',
 'Новый Года',
 'врач',
 'гаишник',
 'работа',
 'учитель',
 'студент',
 'зять',
 'тёща',
 'чукча',
 'Шерлока Холмс',
 'школа',
 'Штирлиц']

In [31]:
b = ""
c = "" # оперативка

И пишем функцию, которая будет их доставать.

In [256]:
def famous_tags(text):
  what_we_found = []
  for i in my_tags:
    if str(i) in text:
      what_we_found.append(str(i))
  return what_we_found

Так как у нас в итоге будут дважды доставаться теги из анекдотов и какие-то из них могут пересекаться, напишем мини-функцию удаления повторок.

In [44]:
def all_tags(a, b):
  c = a + b
  return list(set(c))

## Часть 5. Ужасное вытаскиевание тегов.

_Казалось бы_

Всё просто
1. Вытаскиваем из БД индексы, тексты и обработанные тексты
2. Применяем на них функции вытаскивания тегов
3. Записываем результат в БД

_Но ничего подобного_

**ПРОБЛЕМЫ:** 
Запускать Stanza на каждом анекдоте - умереть можно, либо оперативка кончится (Google Colab), либо надо держать комп сутки(?) работающим.
Потому использовался обходной обходной путь кривого вытаскивания из всего текста с дальнейшей мучительной отладкой. Но результат получен!

In [34]:
dict_of_all_tags = {} # вида тег:id
writing_to_db_tags = [] # там туплы вида (tag,)
writing_to_db_text_to_tag = [] # там туплы вида (id_anec, id_tag)

In [35]:
import sqlite3

In [36]:
just_texts_query = """
SELECT anecs.id, anecs.text, anecs.lemtext FROM anecs
"""

In [38]:
conn = sqlite3.connect("le_petit_db.db")
cur = conn.cursor()
cur.execute(just_texts_query, )
information = cur.fetchall()
information[0]

(1,
 'Впервые за 28 лет депутатам придётся жить, лечиться, учить своих детей в условиях, которые они все эти годы создавали.',
 'впервые 28 год депутат приходиться жить лечиться учить свой ребенок условие который весь год создавать')

In [227]:
len(information)

5963

Придуманный способ сомнительного уровня элегантности: **между каждыми анекдотами вставлять каую-то именованную сущность, которая нигде не встречалась, чтобы затем анализировать её как "слепой" разделитель**

In [127]:
text = ""
for i in information:
    text += i[1]
    text += " . Мухаммед стоит. "
len(text)

935288

Обрабатываем весь текст (около получаса уходит на 1 раз)

In [129]:
nament_tags_M = named_entity_tags(text) # ОООООООООООООООООООООохм

2022-03-29 17:03:48 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| ner       | wikiner   |

2022-03-29 17:03:48 INFO: Use device: cpu
2022-03-29 17:03:48 INFO: Loading: tokenize
2022-03-29 17:03:48 INFO: Loading: ner
2022-03-29 17:03:52 INFO: Done loading processors!


In [130]:
# Какова погрешность?
count_B = 0
for i in nament_tags_M:
    if i == 'мухаммед':
        count_B += 1
count_B

5960

### Важное замечание по поводу тегов. 
Любыми способами выделяется очень много "мусорных" тегов, не хотим их ни записывать ни анализировать потом => делаем доп. условие на запись в бд - только теги встретившиеся больше 2 раз.

In [162]:
print("Всего выделилось сущностей", len(nament_tags_M))

Всего выделилось сущностей 14728


In [169]:
nament_tags_M_unique = set(nament_tags_M)
print("Из них всего различных", len(nament_tags_M_unique))

Из них всего различных 4127


### Способ решения с мусорными тегами.

In [ ]:
a = {1:2, 3:4, 5:0}
f

In [244]:
statistics_on_ne_tags = {}
for i in nament_tags_M:
    if i in statistics_on_ne_tags:
        statistics_on_ne_tags[i] += 1
    else:
        statistics_on_ne_tags[i] = 1
#{k: v for k, v in sorted(statistics_on_ne_tags.items(), key=lambda item: item[1])}
#statistics_on_ne_tags_but_cooler_1 = {}
statistics_on_ne_tags_but_cooler_2 = []
for el in statistics_on_ne_tags:
    if statistics_on_ne_tags[el] > 2:
#        statistics_on_ne_tags_but_cooler[el] = statistics_on_ne_tags[el]
        statistics_on_ne_tags_but_cooler_2.append(el)

In [186]:
print("В усовершенствованном списке", len(statistics_on_ne_tags_but_cooler), "тегов")

В усовершенствованном списке 496 тегов


### Теперь отладка по индексам

Так как Станза работала не идеально и пропустила некоторые вхождения сущностей-разделителей => долго и больно придумываем, как сделать так, чтобы индексы не поехали. В итоге решается следующей функцией.

In [246]:
# ладно, сделаем словрь туплов (номер анека, список тагов)
bleat = []
number_of_anec = 1
small_listic_of_ne_tags = []
for i in nament_tags_M:
    if number_of_anec < 5964:
        if i == 'мухаммед' or 'мухаммед' in i:
            #print('будда встретилось')
            bleat.append(tuple((number_of_anec, small_listic_of_ne_tags)))
            number_of_anec += 1
            small_listic_of_ne_tags = []
        else:
            #print(number_of_anec)
            #print('будда не встретилось')
            if i in statistics_on_ne_tags_but_cooler_2:
                small_listic_of_ne_tags.append(i)
    else:
        break
print(bleat)
    

[(1, []), (2, ['сталин', 'москва', 'минск', 'киев', 'германия']), (3, []), (4, []), (5, ['мерседес']), (6, []), (7, ['сечин', 'миллер']), (8, ['казахстан']), (9, ['россия']), (10, []), (11, ['миллер']), (12, ['россия']), (13, ['россия']), (14, ['запад', 'россия', 'чубайс']), (15, ['новый года']), (16, ['covid', 'россия']), (17, []), (18, ['россия']), (19, ['оруэлл']), (20, ['путин', 'талибан']), (21, ['пепси', 'макдональдс', 'кока-кола', 'американец']), (22, []), (23, ['россия']), (24, []), (25, ['франция', 'россия', 'франция', 'россия']), (26, ['рф', 'apple']), (27, []), (28, ['янукович']), (29, ['москва', 'россия']), (30, ['россия', 'жириновский', 'россия']), (31, ['монако']), (32, ['россия']), (33, []), (34, []), (35, []), (36, []), (37, []), (38, ['роскомнадзор']), (39, []), (40, []), (41, []), (42, []), (43, []), (44, ['рф']), (45, ['киев', 'путин']), (46, []), (47, ['россия']), (48, []), (49, ['китай', 'россия']), (50, ['ссср', 'россия']), (51, ['путин']), (52, ['казахстан', 'каз

In [248]:
information[5939]

(5940,
 'Пьём в интернациональной компании, спрашиваю что-то у немца. Всех убила фраза поляка: - Я немного боюсь, когда русские и немцы о чём-то договариваются. В последний раз после этого не стало Польши.',
 'пить интернациональный компания спрашивать что то немец убивать фраза поляк немного бояться русский немец что то договариваться последний это становиться польша')

In [ ]:
# *, сколько сил * потрачено, но сработало

Пишем функцию, которая на фход принимает элементы из information и bleat а выдаёт списки тэгов и соотношений.

In [272]:
dict_of_all_tags = {}
writing_to_db_tags = [] # там туплы вида (tag,) 
writing_to_db_text_to_tag = [] # там туплы вида (id_anec, id_tag)

In [281]:
def all_text_processing(info, NE):
  text_id = info[0]
  list_of_NE_tags = NE[1]
  list_of_other_tags = famous_tags(info[2])
  all_tags_here = all_tags(list_of_NE_tags, list_of_other_tags)
  for i in all_tags_here: # Индексирование БД
    if i not in dict_of_all_tags:
      dict_of_all_tags[i] = len(dict_of_all_tags) + 1
      writing_to_db_tags.append(tuple((i,len(writing_to_db_tags)+1))) # держится на честном слове
    writing_to_db_text_to_tag.append(tuple((text_id, dict_of_all_tags[i])))

In [259]:
from tqdm.auto import tqdm

### Запустили обработку и вытащили все 'хорошие' теги.

In [282]:
for i in tqdm(range(len(information))):
    all_text_processing(information[i], bleat[i])

### Записали теги и связи между тегами и анекдотами в БД!

In [269]:
conn = sqlite3.connect("le_petit_db.db")
cur = conn.cursor()
cur.executemany(
            """
            INSERT INTO tags 
                (tag)
                VALUES (?)
            """, (
                writing_to_db_tags)
        )
cur.executemany(
            """
            INSERT INTO text_to_tag 
                (id_anec, id_tag)
                VALUES (?, ?)
            """, (
                writing_to_db_text_to_tag)
        )

conn.commit()
conn.close()

In [286]:
conn.close()

На этом этапе у меня стала locked база данных)0

In [288]:
# Код, как удалять
conn = sqlite3.connect('le_petit_db_save_me.db')
cur = conn.cursor()
cur.execute(
    'DELETE FROM tags WHERE tag != ""'
    )
conn.commit()
conn.close()

In [290]:
conn = sqlite3.connect('le_petit_db_save_me.db')
cur = conn.cursor()
cur.executemany(
            """
            INSERT INTO tags 
                (tag, id_tag)
                VALUES (?,?)
            """, (
                writing_to_db_tags)
        )
conn.commit()
conn.close()